#### **Attention is All You Need (NIPS 2017)** 실습
* 본 코드는 기본적으로 **Transformer** 논문의 내용을 최대한 따릅니다.
    * 본 논문은 **딥러닝 기반의 자연어 처리** 기법의 기본적인 구성을 이해하고 공부하는 데에 도움을 줍니다.
    * 2020년 기준 가장 뛰어난 번역 모델들은 본 논문에서 제안한 **Transformer 기반의 아키텍처**를 따르고 있습니다.
* 코드 실행 전에 **[런타임]** → **[런타임 유형 변경]** → 유형을 **GPU**로 설정합니다.

In [1]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 토큰화
spacy_de = spacy.load('de_core_news_sm') # 독일어 토큰화

c:\Users\chlje\anaconda3\envs\dl39v1\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\chlje\anaconda3\envs\dl39v1\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'de_core_news_sm' (3.4.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
# 토큰화 기능 demo
tokenized = spacy_en.tokenizer('I am a graduate student.')

for i, token in enumerate(tokenized):
    print(f'index {i}: {token.text}')

index 0: I
index 1: am
index 2: a
index 3: graduate
index 4: student
index 5: .


* 영어(English) 및 독일어(Deutsch) **토큰화 함수** 정의

In [3]:
# 독일어(Deutsch) 문장을 토큰화 하는 함수 (순서를 뒤집지 않음)
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

# 영어(English) 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

* **필드(field)** 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시합니다.
* Seq2Seq 모델과는 다르게 <b>batch_first 속성의 값을 True로 설정</b>합니다.
    - transformer에선 sequence보다 batch가 먼저오도록 하는 경우가 일반적이라고 함. 
* 번역 목표
    * 소스(SRC): 독일어
    * 목표(TRG): 영어

In [4]:
from torchtext.data import Field, BucketIterator

SRC = Field(
    tokenize=tokenize_de, 
    init_token='<sos>',
    eos_token='<eos>',
    lower=True, # 독일어의 경우 떼주는게 낫지 않은가? 근데 이렇게 lower해주는게 일반적이라고 한다. 
    batch_first=True
    )

TRG = Field(
    tokenize=tokenize_en, 
    init_token='<sos>',
    eos_token='<eos>',
    lower=True, 
    batch_first=True
    )

* 대표적인 영어-독어 번역 데이터셋인 **Multi30k**를 불러옵니다.

    - 위에 정의했던 필드 전처리를 수행함. 

In [5]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))

In [6]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [7]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


* **필드(field)** 객체의 **build_vocab** 메서드를 이용해 영어와 독어의 단어 사전을 생성합니다.
  * **최소 2번 이상** 등장한 단어만을 선택합니다.

In [8]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7853
len(TRG): 5893


In [9]:
# 토큰의 index를 확인 

print(TRG.vocab.stoi["abcabc"]) # 없는 단어: 0
print(TRG.vocab.stoi[TRG.pad_token]) # 패딩(padding): 1
print(TRG.vocab.stoi["<sos>"]) # <sos>: 2
print(TRG.vocab.stoi["<eos>"]) # <eos>: 3
print(TRG.vocab.stoi["hello"])
print(TRG.vocab.stoi["world"])

0
1
2
3
4112
1752


* 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야 합니다.
    * 따라서 하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋습니다.
        - 한 배치에 사용할 문장들의 수. 즉, 128개씩 끊어서 문장길이를 재조정해줌. 
        - 한 문장에서 패딩의 수가 너무 많지 않게 함. 그러면서도 충분한 길이를 유지 
    * 이를 위해 `BucketIterator`를 사용합니다.
    * **배치 크기(batch size)**: 128

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [11]:
BATCH_SIZE = 128

# 일반적인 데이터 로더(data loader)의 iterator와 유사하게 사용 가능
train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size=BATCH_SIZE,
    device=device,
    )

In [12]:
for i, batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg

    print(f"첫 번째 배치 크기: {src.shape}")

    # 현재 배치에 있는 하나의 문장에 포함된 정보 출력
    for i in range(src.shape[1]):
        print(f"인덱스 {i}: {src[0][i].item()}") # 여기에서는 [Seq_num, Seq_len]

    # 첫 번째 배치만 확인
    break

첫 번째 배치 크기: torch.Size([128, 36])
인덱스 0: 2
인덱스 1: 18
인덱스 2: 537
인덱스 3: 9
인덱스 4: 8
인덱스 5: 16
인덱스 6: 7
인덱스 7: 173
인덱스 8: 38
인덱스 9: 1185
인덱스 10: 10
인덱스 11: 5
인덱스 12: 13
인덱스 13: 7
인덱스 14: 6
인덱스 15: 71
인덱스 16: 255
인덱스 17: 38
인덱스 18: 17
인덱스 19: 7009
인덱스 20: 4
인덱스 21: 3
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1
인덱스 25: 1
인덱스 26: 1
인덱스 27: 1
인덱스 28: 1
인덱스 29: 1
인덱스 30: 1
인덱스 31: 1
인덱스 32: 1
인덱스 33: 1
인덱스 34: 1
인덱스 35: 1


첫 배치의 문장의길이: 36이 되는거고, 

뒤의 단어들은 1의 패딩토큰으로 채워지게 됨. 

#### **Multi Head Attention 아키텍처**

* 어텐션(attention)은 <b>세 가지 요소</b>를 입력으로 받습니다.
    * <b>쿼리(queries)</b>
    * <b>키(keys)</b>
    * <b>값(values)</b>
    * 현재 구현에서는 Query, Key, Value의 차원이 모두 같습니다.
* 하이퍼 파라미터(hyperparameter)
    * **hidden_dim**: 하나의 단어에 대한 임베딩 차원
    * **n_heads**: 헤드(head)의 개수 = scaled dot-product attention의 개수
    * **dropout_ratio**: 드롭아웃(dropout) 비율

In [13]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # embedding 차원
        self.n_heads = n_heads # 헤드의 갯수. = 서로 다른 attention 컨셉의 수. 
        self.head_dim = hidden_dim // n_heads # 각 헤드의 임베딩 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC layer
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC layer
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC layer

        self.fc_o = nn.Linear(hidden_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device) # QKV 계산에 나오는 루트 d_k
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]
        # query: [batch_size, query_len, hidden_dim]
        # key: [batch_size, key_len, hidden_dim]
        # value: [batch_size, value_len, hidden_dim]

        Q = self.fc_q(query)
        K = self.fc_q(key)
        V = self.fc_q(value)
        # Q: [batch_size, query_len, hidden_dim]
        # K: [batch_size, key_len, hidden_dim]
        # V: [batch_size, value_len, hidden_dim]

        # hidden_dim → n_heads X head_dim 형태로 변형
        # n_heads(h)개의 서로 다른 어텐션(attention) 컨셉을 학습하도록 유도
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3) # -1 부분이 query_len
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        # Q: [batch_size, n_heads, query_len, head_dim]
        # K: [batch_size, n_heads, key_len, head_dim]
        # V: [batch_size, n_heads, value_len, head_dim]

        # Attention Energy 계산
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        # energy: [batch_size, n_heads, query_len, key_len]

        # 마스크(mask)를 사용하는 경우
        if mask is not None:
            # 마스크(mask) 값이 0인 부분을 -1e10으로 채우기 # --> softmax의 출력이 0%에 가까워지게 함. 
            energy = energy.masked_fill(mask==0, -1e10) 
        
        
        # 어텐션(attention) 스코어 계산: 각 단어에 대한 확률 값
        attention = torch.softmax(energy, dim=1)
        # attention: [batch_size, n_heads, query_len, key_len]

        # 여기에서 Scaled Dot-Product Attention을 계산
        x = torch.matmul(self.dropout(attention), V)
        # x: [batch_size, n_heads, query_len, head_dim]

        x = x.permute(0, 2, 1, 3).contiguous() # contiguous하면 마치 view 대신 copy만들듯 permute된 tensor를 메모리에 박아버림
        # x: [batch_size, query_len, n_heads, head_dim]

        x = x.view(batch_size, -1, self.hidden_dim)
        # x: [batch_size, query_len, hidden_dim]

        x = self.fc_o(x)
        # x: [batch_size, query_len, hidden_dim]

        return x, attention